In [3]:
import torch
import fsspec
from transformers import AutoTokenizer, AutoModelForCausalLM
import tempfile
import os
import shutil

In [1]:
model_path = "https://sominmodelstorage.blob.core.windows.net/mediummodel/model%2Fmedium-tech%2F"

# Replace the placeholders with your own values
storage_account_name = "sominmodelstorage"
container_name = "mediummodel"
model_directory = "model/medium-tech"  # The directory in the container where the model files are stored

In [7]:
from azure.storage.blob import BlobServiceClient, PublicAccess

connection_string="DefaultEndpointsProtocol=https;AccountName=sominmodelstorage;AccountKey=Rd/Ds7pibf6JbKOgOqi4kubWsgSbcvoopwt76BQqaOsF0zosupNNJmeGQ0fzbmmowHwy6393i7Xc+ASt7HOH9w==;EndpointSuffix=core.windows.net"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# Allow public read access for both the container and its blobs
container_client.set_container_access_policy(signed_identifiers={}, public_access=PublicAccess.Container)
print(f"Public access level of container '{container_name}' and blobs set to 'public'.")

Public access level of container 'mediummodel' and blobs set to 'public'.


In [8]:
# Create a file system that maps to Azure Blob Storage
fs = fsspec.filesystem("az", account_name=storage_account_name)

# Create a temporary directory to store the downloaded model files
temp_dir = tempfile.mkdtemp()

# Download all model files from the container to the temporary directory
for file in fs.glob(f"az://{container_name}/{model_directory}/*"):
    with fs.open(file, "rb") as src, open(os.path.join(temp_dir, os.path.basename(file)), "wb") as dst:
        shutil.copyfileobj(src, dst)

# Load the tokenizer and model from the temporary directory
ft_tokenizer = AutoTokenizer.from_pretrained(temp_dir)
ft_model = AutoModelForCausalLM.from_pretrained(temp_dir)

# Now you can use the tokenizer and model for inference

# Clean up the temporary directory
shutil.rmtree(temp_dir)


In [11]:
text = "Data scientists use VS Code as their tool of choice because"

ft_input_ids = ft_tokenizer.encode(text, return_tensors='pt')

output = ft_model.generate(ft_input_ids, attention_mask = torch.ones_like(ft_input_ids), pad_token_id=ft_tokenizer.eos_token_id,
                              max_length=100, do_sample=True)# num_beams=2, no_repeat_ngram_size=3, early_stopping=False)

print(ft_tokenizer.decode(output[0], skip_special_tokens=True))

Data scientists use VS Code as their tool of choice because VS codes words. This means that the VS Code tool is used to determine the vocabulary for different words in the text of the text. This means that the VS codes the text of the text of the text (the VS code code from the text) are compared to the words (the words from the text).

This is as a result that in most cases, the data scientists use the words from the text as if they were
